# Домашнее задание "Бинарная классификация изображений и решение проблемы дисбаланса классов"

## Введение
В этом домашнем задании вы будете работать с задачей бинарной классификации изображений. Мы будем использовать датасет "Cats vs Dogs", чтобы создать модель, способную различать изображения кошек и собак. Тренировочный набор данных содержит 10 000 изображений кошек и 10 000 изображений собак. В тестовом наборе содержится по 2 500 картинок каждого класса.

Цель домашнего задания — закрепить знания о том, как загружать и предобрабатывать данные, определять и обучать нейронные сети, оценивать их эффективность и визуализировать результаты.

## Импорт необходимых библиотек
Для решения поставленной задачи вам понадобятся библиотеки для работы с числовыми данными, обучения нейронных сетей и отрисовки графиков. В этой части кода импортируйте необходимые библиотеки, установите seed для воспроизводимости результатов и определите устройство для работы — cpu или cuda.

In [ ]:
# YOUR CODE HERE

## Задание №1: Загрузка и предобработка данных
Теперь вам необходимо выполнить загрузку и предобработку данных. В качестве бейзлайна можете использовать следующие аугментации (обязательно поэкспериментируйте с другими!):
- изменение размера изображения до 224х224;
- случайное горизонтальное отражение;
- случайное изменение яркости, контраста, насыщенности и оттенка;
- преобразование в тензор.

Для тестового набора данных используйте только изменение размера до 224х224 и преобразование в тензор.

In [ ]:
# Преобразования для обучающего набора данных
train_transform = ... # YOUR CODE HERE

# Преобразования для тестового набора данных
test_transform = ... # YOUR CODE HERE

Датасет для скачивания вы можете найти по [ссылке](https://www.kaggle.com/datasets/salader/dogs-vs-cats?select=test). После скачивания откройте Google Drive (Google Диск) и загрузите архив в удобную для вас директорию.

In [ ]:
# Подключение Google Диска
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Разархивирование скачанного архива (при необходимости замените пути)
!unzip -q dogs-vs-cats.zip "train/*" "test/*" -d dogs_vs_cats

Убедитесь, что в разделе `Файлы` у вас появилась папка следующего вида:
```
dogs_vs_cats/
├── train/
│   ├── cats/
│   └── dogs/
└── test/
    ├── cats/
    └── dogs/
```
После этого можно перейти к формированию датасета. При работе с данными такой структуры применяется функция `ImageFolder` из `torchvision.datasets`. Примените заданные трансформы. При необходимости измените пути к данным.

In [ ]:
# Загрузка обучающего набора данных
train_data = ... # YOUR CODE HERE
test_data = ... # YOUR CODE HERE

## Создание несбалансированной выборки

Для усложнения задачи создадим несбалансированную версию датасета. В качестве редуцируемого класса выберем "cats".

In [ ]:
import os
import random
from torchvision import datasets
from torch.utils.data import Subset

def create_imbalanced_dataset(dataset, percentage=0.1):
    # Разделим индексы по классам
    cats_indices = [i for i, (_, label) in enumerate(dataset) if dataset.classes[label] == 'cats']
    dogs_indices = [i for i, (_, label) in enumerate(dataset) if dataset.classes[label] == 'dogs']
    # Выберем 10% от класса 'cats'
    selected_cats_indices = random.sample(cats_indices, int(len(cats_indices) * percentage))
    final_indices = selected_cats_indices + dogs_indices

    return Subset(dataset, final_indices)

In [ ]:
# Создание несбалансированного датасета (10% класса 'cats')
imbalanced_train_data = create_imbalanced_dataset(train_data, percentage=0.1)

Теперь создайте загрузчики данных (DataLoader) для тренировочного и тестового наборов данных.  
Не забудьте про перемешивание обучающего набора данных перед каждой эпохой. Также задайте параметры *worker_init_fn* и *generator* для воспроизводимости.

In [ ]:
imbalanced_train_loader = ... # YOUR CODE HERE
test_loader = ... # YOUR CODE HERE

## Задание №2: Обучение модели на несбалансированных данных
Выберите модель и обучите ее на несбалансированных данных. Вы можете взять функцию для обучения из семинара, а лучше напишите свою. Не забывайте сохранять веса обученных моделей.

После обучения модели посмотрите отчет о классификации и матрицу ошибок, используя `classification_report` и `confusion_matrix` из `sklearn.metrics`. Что можно сказать о полученных метриках?

In [ ]:
# YOUR CODE HERE

## Решение проблемы дисбаланса классов
Чтобы модель могла лучше предсказывать малопредставленные классы, необходимо использовать специальные методы, для адаптации процесса обучения. Рассмотрим такие подходы.


## Задание №3: Resampling
Resampling включает в себя ве техники — **under**sampling и **over**sampling.

**Undersampling** — это метод, направленный на борьбу с дисбалансом классов, при котором количество объектов в более часто встречающихся классах уменьшается, чтобы выровнять количество объектов в каждом классе.
Этот подход эффективен, когда объем данных велик, и удаление части данных не приведет к значительной потере информации. Его основное преимущество — это сокращение времени и ресурсов, необходимых для обучения модели. Однако минус заключается в том, что важная информация может быть потеряна, если вместе с избыточными данными удаляются редкие события или аномалии.

**Oversampling**, наоборот, увеличивает количество примеров в менее представленном классе за счет дублирования существующих примеров или генерации новых синтетических данных.
Этот метод сохраняет всю информацию исходного набора данных, увеличивая вероятность того, что модель научится распознавать менее частые классы.

![resampling](https://blog.strands.com/hs-fs/hubfs/Screenshot%202019-07-18%20at%2014.15.15.png?width=1800&name=Screenshot%202019-07-18%20at%2014.15.15.png)

Реализуйте оба метода, используя `WeightedRandomSampler` из `torch.utils.data`. Вычислите веса для классов на основе обратной частоты их появления. Передайте список весов для каждого элемента в наборе данных в качестве аргумента `WeightedRandomSampler`. Полученный семплер используйте при создании DataLoader.

### Undersampling

In [ ]:
# YOUR CODE HERE

### Oversampling

In [ ]:
# YOUR CODE HERE

Сравните метрики с теми, что были получены при первоначальнои обучении. Удалось ли повысить качество модели?

In [ ]:
# YOUR CODE HERE

## Задание №4: Взвешивание классов
Другой подход, используемый при дисбалансе — взвешивание классов. Этот метод позволяет уменьшить предвзятость модели к более представленным классам, назначая больший вес ошибкам на менее представленных классах. Веса классов обычно вычисляются на основе обратной зависимости от частоты классов в данных. Полученные веса используются в качестве аргмента функции потерь.

Взвесьте классы согласно этой формуле:
$$ w_i = \frac{1}{n_i} $$
где:  
$w_i$ — вес для $i$-го класса;  
$n_i$ — число объектов $i$-го класса.

Полученые веса передайте в качестве аргумента функции потерь и запустите обучение.

In [ ]:
# YOUR CODE HERE

Снова проверьте метрики на тестовом наборе данных.

In [ ]:
# YOUR CODE HERE

## Задание №5: Focal Loss
Focal Loss — это функция потерь, используемая в задачах классификации для решения проблемы несбалансированных классов. Focal Loss модифицирует стандартную кросс-энтропийную потерю так, чтобы уменьшить влияние легко классифицируемых объектов на обучение модели и увеличить влияние "сложных", мало представленных объектов.
$$ FL(p_t) = -\sum_{c=1}^C \alpha_c (1 - p_{t,c})^\gamma \log(p_{t,c}) $$
где:  
$p_t$ — вероятность принадлежности к целевому классу;  
$\alpha_c$ — весовой коэффициент для класса $c$;  
$\gamma$ — параметр, регулирующий степень уменьшения веса легко классифицируемых объектов.

Случай $\gamma=0$ соотвествует кросс-энтропийной функции потерь. Значение параметра $\gamma>0$ уменьшает относительные потери для хорошо классифицированных объектов (верно предсказанных с высокой вероятностью), уделяя больше внимания сложным, неправильно классифицированным объектам.

![focal_loss](https://i2.wp.com/miro.medium.com/1*zvntw5mnlWLR7zUMKCS_AA.png)

Реализуйте функцию Focal Loss и задайте ее в качестве функции потерь при обучении модели.

In [ ]:
# YOUR CODE HERE

Посчитайте метрики на тестовом наборе данных. Какой из подходов дал наилучший результат?

In [ ]:
# YOUR CODE HERE

## Задание №6: Выбор лучшей модели и получение предсказаний
Выберите лучшую модель и сделайте предсказания на тестовом наборе данных. Визуализируйте полученные результаты.

In [ ]:
# YOUR CODE HERE

## Заключение
В этом домашнем задании вы закрепили знания по подготвке данных и обучению нейронных сетей, а также научились решать проблему несбалансированного датасета.
Полученные знания и навыки позволят вам успешно решать задачи классификации изображений.